# Goals

### - Improve our original estimate of the log error by using clustering methodologies.
### - Build a model to accurately predict the log error

# Hypotheses
What do we think causes log error:
- Location: certain geographic areas may be harder to accurately predict than others. Log error may differ by County. 
- Price: higher prices may have more variance which makes them harder to predict
- Age of Property: newer homes may be similarily built, thus easier to predict price. Older property values may have more variance, thus making them harder to predict
- Amenities: fireplace, pool, hot tub, and garage are nice amenities that not all properties include. These amenities may be driving log error by making property values harder to predict.
- Data density: geographic areas with more data points (higher data density) will have lower log errors. Geogrphaic areas with less data points (lower data density) will have higher log errors. The more data we have in a given geographic area, the lower we would expect those log errors to be.

## Set up Environment

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
#import acquire
import prep
%matplotlib inline
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

# Acquire

### Here we acquire our data from the SQL database. We have already chosen to filter on the following conditions:
- Only include properties containing latitude and longitude values.
- Only include properties with a transaction in 2017 (including the most recent transaction date to avoid duplicates).
- Only include single unit properties. 
  - Single unit property: a free standing property in which the owner also owns the land beneath the property. A single unit property is built for the use of one family (i.e. does not include duplexes, triplexes, etc.)

In [2]:
data = pd.read_csv('query_result.csv')

Preview data:

In [3]:
data.shape

(52357, 61)

In [4]:
data.head(5)

,parcelid,id,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,logerror,transactiondate
0,14297519,1727539,NaN,NaN,NaN,3.5,4,NaN,NaN,3.5,...,485713.0,1023282.0,2016,537569.0,11013.72,NaN,NaN,6.059063e+13,0.025595,2017-01-01
1,17052889,1387261,NaN,NaN,NaN,1.0,2,NaN,NaN,1.0,...,88000.0,464000.0,2016,376000.0,5672.48,NaN,NaN,6.111001e+13,0.055619,2017-01-01
2,14186244,11677,NaN,NaN,NaN,2.0,3,NaN,NaN,2.0,...,85289.0,564778.0,2016,479489.0,6488.30,NaN,NaN,6.059022e+13,0.005383,2017-01-01
3,12177905,2288172,NaN,NaN,NaN,3.0,4,NaN,8.0,3.0,...,108918.0,145143.0,2016,36225.0,1777.51,NaN,NaN,6.037300e+13,-0.103410,2017-01-01
4,12095076,781532,1.0,NaN,NaN,3.0,4,NaN,9.0,3.0,...,276684.0,773303.0,2016,496619.0,9516.26,NaN,NaN,6.037461e+13,-0.001011,2017-01-01


In [5]:
data.columns

Index(['parcelid', 'id', 'airconditioningtypeid', 'architecturalstyletypeid',
       'basementsqft', 'bathroomcnt', 'bedroomcnt', 'buildingclasstypeid',
       'buildingqualitytypeid', 'calculatedbathnbr', 'decktypeid',
       'finishedfloor1squarefeet', 'calculatedfinishedsquarefeet',
       'finishedsquarefeet12', 'finishedsquarefeet13', 'finishedsquarefeet15',
       'finishedsquarefeet50', 'finishedsquarefeet6', 'fips', 'fireplacecnt',
       'fullbathcnt', 'garagecarcnt', 'garagetotalsqft', 'hashottuborspa',
       'heatingorsystemtypeid', 'latitude', 'longitude', 'lotsizesquarefeet',
       'poolcnt', 'poolsizesum', 'pooltypeid10', 'pooltypeid2', 'pooltypeid7',
       'propertycountylandusecode', 'propertylandusetypeid',
       'propertyzoningdesc', 'rawcensustractandblock', 'regionidcity',
       'regionidcounty', 'regionidneighborhood', 'regionidzip', 'roomcnt',
       'storytypeid', 'threequarterbathnbr', 'typeconstructiontypeid',
       'unitcnt', 'yardbuildingsqft17', 'yardb

# Prepare
### There are several rows and columns that are missing data. We need to treat null values by dropping columns, dropping rows, or imputing missing values.

### Preview nulls

This function shows the amount of rows that have null values for each column. We use this to check if there are any columns that are droppable due to a large portion of nulls.

In [6]:
prep.nulls_by_col(data)

,num_rows_missing,pct_rows_missing
parcelid,0,0.000000
id,0,0.000000
airconditioningtypeid,38738,0.739882
architecturalstyletypeid,52287,0.998663
basementsqft,52310,0.999102
bathroomcnt,0,0.000000
bedroomcnt,0,0.000000
buildingclasstypeid,52357,1.000000
buildingqualitytypeid,18692,0.357011
calculatedbathnbr,162,0.003094


This function shows the amount of rows that are missing a certain number of columns. We use this to check if there are any rows that are missing so many columns, they are not worth keeping.
We will drop columns first, and then come back and check these rows to see how they were affected.

In [7]:
prep.nulls_by_row(data)

,num_cols_missing,pct_cols_missing,num_rows
0,19,31.147540983606557,2
1,20,32.78688524590164,10
2,21,34.42622950819672,83
3,22,36.0655737704918,261
4,23,37.704918032786885,480
5,24,39.34426229508197,534
6,25,40.98360655737705,2910
7,26,42.62295081967213,5921
8,27,44.26229508196721,7934
9,28,45.90163934426229,15854


### Run data through our prep function to acheive the following:
- fills nulls in taxdelinquencyflag with 'N'
- fills nulls with zero for fireplacecnt, garagecarcnt, poolcnt, and taxdelinquencyyear
- drops 27 columns due to excess nulls and irrelevency
- encodes the fips columns
- data shape trasnformed from 52357 rows x 61 columns to 46756 rows x 53 columns

In [8]:
data = prep.prep_zillow(data)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52357 entries, 0 to 52356
Data columns (total 61 columns):
parcelid                        52357 non-null int64
id                              52357 non-null int64
airconditioningtypeid           13619 non-null float64
architecturalstyletypeid        70 non-null float64
basementsqft                    47 non-null float64
bathroomcnt                     52357 non-null float64
bedroomcnt                      52357 non-null int64
buildingclasstypeid             0 non-null float64
buildingqualitytypeid           33665 non-null float64
calculatedbathnbr               52195 non-null float64
decktypeid                      389 non-null float64
finishedfloor1squarefeet        4371 non-null float64
calculatedfinishedsquarefeet    52250 non-null float64
finishedsquarefeet12            52084 non-null float64
finishedsquarefeet13            0 non-null float64
finishedsquarefeet15            0 non-null float64
finishedsquarefeet50            4371 n

### Preview nulls again

In [9]:
prep.nulls_by_col(data)

,num_rows_missing,pct_rows_missing
parcelid,0,0.000000
id,0,0.000000
bathroomcnt,0,0.000000
bedroomcnt,0,0.000000
buildingqualitytypeid,16372,0.350158
decktypeid,46481,0.994118
calculatedfinishedsquarefeet,60,0.001283
fips,0,0.000000
fireplacecnt,0,0.000000
fullbathcnt,104,0.002224


# Explore
To do:

- Check our hypotheses
- 
- 
- 